In [ ]:
!pip install tensorflow

In [257]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

cleaning + preprocessing data 

In [279]:
stocks_df = pd.read_csv('stocks.csv')

/var/folders/v6/lhv05hmx5v94886x_jxd39s00000gn/T/ipykernel_20597/1353134829.py:1: DtypeWarning: Columns (9,49,57) have mixed types. Specify dtype option on import or set low_memory=False.
  stocks_df = pd.read_csv('stocks.csv')


In [280]:
stocks_df = stocks_df.drop(columns=['PERMCO', 'COMNAM', 'HEXCD', 'ISSUNO', 'TRDSTAT', 'SECSTAT', 'DISTCD'], axis=1)

In [281]:
missing_percentage = stocks_df.isnull().sum() * 100 / len(stocks_df)
columns_to_drop = missing_percentage[missing_percentage > 90].keys()
stocks_df = stocks_df.drop(columns=columns_to_drop, axis=1)
stocks_df = stocks_df.dropna()

In [282]:
to_encode = stocks_df.select_dtypes(include=['object']).columns
not_encode = stocks_df.select_dtypes(exclude=['object']).columns
not_encode = not_encode.drop(labels=['PRC'])

In [318]:
stocks_df.columns

Index(['PERMNO', 'date', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP', 'TICKER',
       'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'HSICCD', 'CUSIP', 'HSICMG',
       'HSICIG', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT',
       'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD', 'RETX', 'vwretd', 'vwretx',
       'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')

In [ ]:
def create_sequences(data, seq_length):
    sequences = []
    targets = []
    
    for i in range(len(data) - seq_length):
        sequence = data[i:(i + seq_length)]
        target = data[i + seq_length][3]  
        sequences.append(sequence)
        targets.append(target)
    
    return np.array(sequences), np.array(targets)

In [283]:
X = stocks_df[['PERMNO', 'date', 'SHRCD', 'NCUSIP', 'EXCHCD', 'SICCD', 'TICKER',
       'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'HSICCD', 'CUSIP', 'HSICMG',
       'HSICIG', 'BIDLO', 'ASKHI', 'VOL', 'RET', 'BID', 'ASK', 'SHROUT',
       'CFACPR', 'CFACSHR', 'OPENPRC', 'NUMTRD', 'RETX', 'vwretd', 'vwretx',
       'ewretd', 'ewretx', 'sprtrn']]

y = stocks_df[['PRC', 'date']]

In [284]:
stocks_df['date'] = pd.to_datetime(stocks_df['date'])

train_start = '2019-01-02'
train_end = '2022-12-30'
test_start = '2023-01-03'
test_end = '2023-12-29'

In [285]:
X_train = X[(X['date'] >= train_start) & (X['date'] <= train_end)]
y_train = y[(y['date'] >= train_start) & (y['date'] <= train_end)]
X_test = X[(X['date'] >= test_start) & (X['date'] <= test_end)]
y_test = y[(y['date'] >= test_start) & (y['date'] <= test_end)]

In [286]:
X_train['date'] = X_train['date'].apply(lambda x: x.replace('-', '')).astype(int) 
X_test['date'] = X_test['date'].apply(lambda x: x.replace('-', '')).astype(int) 

y_train['date'] = y_train['date'].apply(lambda x: x.replace('-', '')).astype(int) 
y_test['date'] = y_test['date'].apply(lambda x: x.replace('-', '')).astype(int) 

/var/folders/v6/lhv05hmx5v94886x_jxd39s00000gn/T/ipykernel_20597/2855478841.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['date'] = X_train['date'].apply(lambda x: x.replace('-', '')).astype(int)
/var/folders/v6/lhv05hmx5v94886x_jxd39s00000gn/T/ipykernel_20597/2855478841.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['date'] = X_test['date'].apply(lambda x: x.replace('-', '')).astype(int)
/var/folders/v6/lhv05hmx5v94886x_jxd39s00000gn/T/ipykernel_20597/2855478841.py:4: SettingWithC

CNN-LSTM hybrid model for stock price prediction

In [290]:
encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(X_train[to_encode])

X_train_encoded = encoder.transform(X_train[to_encode])
X_test_encoded = encoder.transform(X_test[to_encode])

X_train_encoded_dense = X_train_encoded.toarray()
X_test_encoded_dense = X_test_encoded.toarray()

X_train_encoded_df = pd.DataFrame(X_train_encoded_dense, columns=encoder.get_feature_names_out(to_encode))
X_test_encoded_df = pd.DataFrame(X_test_encoded_dense, columns=encoder.get_feature_names_out(to_encode))

X_train_encoded_full = pd.concat([X_train_encoded_df, X_train[not_encode].reset_index(drop=True)], axis=1)
X_test_encoded_full = pd.concat([X_test_encoded_df, X_test[not_encode].reset_index(drop=True)], axis=1)


In [291]:
scaler = StandardScaler()

X_train_scaled = X_train_encoded_full.copy(deep = True)
X_test_scaled = X_test_encoded_full.copy(deep = True)

scaler.fit(X_train_scaled[not_encode])

X_train_scaled[not_encode] = scaler.transform(X_train_scaled[not_encode])
X_test_scaled[not_encode] = scaler.transform(X_test_scaled[not_encode])

input layer 
one-dimensional convolution layer 
pooling layer 
LSTM hidden layer 
full connection layer 

In [292]:
X_train_reshaped = X_train_encoded_full.values.reshape(X_train_encoded_full.shape[0], 1, X_train_encoded_full.shape[1])

In [312]:
model = models.Sequential()
model.add(layers.Conv1D(filters=64, kernel_size=3, padding='same', activation='tanh', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
model.add(LSTM(units=100, activation='tanh'))
model.add(Dense(units=1))

/Users/monishakrothapalli/Documents/GitHub/playlistGen/.conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [313]:
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer, loss='mean_absolute_error', metrics=['mean_absolute_error'])

In [314]:
history = model.fit(X_train_reshaped, y_train, epochs=100, batch_size=64, validation_split=0.1)

train_accuracy = history.history['mean_absolute_error']

for epoch, accuracy in enumerate(train_accuracy):
    print(f"Epoch {epoch + 1}: mean_absolute_error = {accuracy:.4f}")

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 10103756.0000 - mean_absolute_error: 10103756.0000 - val_loss: 10110594.0000 - val_mean_absolute_error: 10110594.0000
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10103790.0000 - mean_absolute_error: 10103790.0000 - val_loss: 10110590.0000 - val_mean_absolute_error: 10110590.0000
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10103971.0000 - mean_absolute_error: 10103971.0000 - val_loss: 10110589.0000 - val_mean_absolute_error: 10110589.0000
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10103890.0000 - mean_absolute_error: 10103890.0000 - val_loss: 10110586.0000 - val_mean_absolute_error: 10110586.0000
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 10104091.0000 - mean_absolute_error: 10104091.0000 - val_loss: 10110584.0000 - val_mean_absolute_error: 10110584.0000
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 10104055.0000 - mean_absolute_error: 10104055.0000 - va

In [315]:
X_test_scaled_array = X_test_scaled.to_numpy()
X_test_scaled_reshaped = X_test_scaled_array.reshape(X_test_scaled_array.shape[0], 1, X_test_scaled_array.shape[1])

predictions = model.predict(X_test_scaled_reshaped)

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [316]:
mae = mean_absolute_error(y_test['PRC'], predictions)
rmse = np.sqrt(mean_squared_error(y_test['PRC'], predictions))
r2 = r2_score(y_test['PRC'], predictions)

print(f'MAE: {mae}, RMSE: {rmse}, R2: {r2}')

MAE: 199.0390944218925, RMSE: 225.26410767150287, R2: -7.231628078127789
